In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [57]:
papers = list(range(20))
leads = list(range(5))
editors = list(range(15))

papers_per_team = 5

c_ik = np.random.rand(len(editors), len(papers))
d = np.random.rand(len(leads), len(papers))
c = np.array([c_ik for _ in leads])

In [51]:
m = gp.Model()

x_ij = m.addVars(editors, leads, vtype=GRB.BINARY, name="editor_lead")
y_jk = m.addVars(leads, papers, vtype=GRB.BINARY, name="lead_paper")
x_ijk = m.addVars(editors, leads, papers, vtype=GRB.BINARY, name="paper_lead_editor")

m.update()

In [53]:
# EACH EDITOR ONLY ASSIGNED TO ONE LEAD / TEAM
editor_one_lead = m.addConstrs(gp.quicksum(x_ij[i, j] for j in leads) == 1 for i in editors)

# EACH PAPER ASSIGNED TO AT LEAST ONE TEAM
paper_one_team = m.addConstrs(gp.quicksum(y_jk[j, k] for j in leads) >= 1 for k in papers)

# EACH TEAM GETS SPECIFIED # OF PAPERS (PARAMETER)
# TODO: is the above constraint redundant when we have this one? (I think so)
paper_count = m.addConstrs(gp.quicksum(y_jk[j, k] for k in papers) == papers_per_team for j in leads)


# RELATE VARIABLES
relate = m.addConstrs(x_ijk[i, j, k] == x_ij[i, j] * y_jk[j, k] for i in editors for j in leads for k in papers)

In [54]:
m.update()

In [60]:
m.setObjective(gp.quicksum(c[j, i, k] * x_ijk[i, j, k] for i in editors for j in leads for k in papers) + gp.quicksum(d[j, k] * y_jk[j, k] for j in leads for k in papers))
m.update()

In [61]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 80 rows, 1675 columns and 550 nonzeros
Model fingerprint: 0x1fbe43c0
Model has 3000 quadratic constraints
Variable types: 0 continuous, 1675 integer (1675 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]

Loaded MIP start from previous solve with objective 51.6235

Presolve removed 40 rows and 0 columns
Presolve time: 0.00s
Presolved: 6040 rows, 3175 columns, 13775 nonzeros
Variable types: 0 continuous, 3175 integer (3175 binary)

Root relaxation: objective 4.601232e+00, 205 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Cur

{(0, 0): <gurobi.Var lead_paper[0,0] (value 0.0)>,
 (0, 1): <gurobi.Var lead_paper[0,1] (value 1.0)>,
 (0, 2): <gurobi.Var lead_paper[0,2] (value 0.0)>,
 (0, 3): <gurobi.Var lead_paper[0,3] (value 0.0)>,
 (0, 4): <gurobi.Var lead_paper[0,4] (value 1.0)>,
 (0, 5): <gurobi.Var lead_paper[0,5] (value 0.0)>,
 (0, 6): <gurobi.Var lead_paper[0,6] (value 0.0)>,
 (0, 7): <gurobi.Var lead_paper[0,7] (value 0.0)>,
 (0, 8): <gurobi.Var lead_paper[0,8] (value 0.0)>,
 (0, 9): <gurobi.Var lead_paper[0,9] (value 0.0)>,
 (0, 10): <gurobi.Var lead_paper[0,10] (value 0.0)>,
 (0, 11): <gurobi.Var lead_paper[0,11] (value 0.0)>,
 (0, 12): <gurobi.Var lead_paper[0,12] (value 1.0)>,
 (0, 13): <gurobi.Var lead_paper[0,13] (value 0.0)>,
 (0, 14): <gurobi.Var lead_paper[0,14] (value 0.0)>,
 (0, 15): <gurobi.Var lead_paper[0,15] (value 1.0)>,
 (0, 16): <gurobi.Var lead_paper[0,16] (value 0.0)>,
 (0, 17): <gurobi.Var lead_paper[0,17] (value 0.0)>,
 (0, 18): <gurobi.Var lead_paper[0,18] (value 0.0)>,
 (0, 19): <gu